### Scribble editor and result display 
#### display of scribbles in slicer, possible manual scribble update in slicer, conversion of scribbles to seeds for graphcut (initially 2D only),invoking graphcut, displaying result of graphcut in slicer, iteration in slicer, save result